This notebook shows how you can use Google Colab with free GPU to get Bert model predictions.
Generally it is nothing special except the times where there is not enough GPU memory to do the task.
To make it possible traing set is being traine in batches. Below there is example  BERT model = 'cased_L-12_H-768_A-12' is used

# Parameters

First you need to pass details of the bert model that you want to use.
You can get these parameters from [this website](https://github.com/google-research/bert). When you right-click on the selected model, you should be able to copy the link which has information about `date` and `name` of the model.

In [ ]:
bert_model_name = 'cased_L-12_H-768_A-12'
bert_model_date = '2018_10_18'
bert_model_path = f'https://storage.googleapis.com/bert_models/{bert_model_date}/{bert_model_name}.zip'
uncased=False

You need to specify directories that will be used in the notebook:
- directory with model data,
- directory with input data,
- output directory.

Path `./drive/My Drive/` points to your Google Drive where you can create folder to be used for storing input and output data.


In [ ]:
model_dir = f'./{bert_model_name}'

data_dir = './drive/My Drive/Colab Notebooks/input'
train_dir = f'{data_dir}/train_fake.csv'
test_dir = f'{data_dir}/test_fake.csv'

out_dir = './drive/My Drive/Colab Notebooks/output'

# Setup

## Mount google drive

Mounting Google Drive is needed to be able to access folders that are inside. After running this cell you will see a link allowing you to authorize your access.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Download model files

In [ ]:
!wget {bert_model_path}
!unzip the file
!unzip {bert_model_name}.zip

## Install libraries

In [ ]:
!pip install keras_bert
!pip install transformers

# Imports

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from keras_bert import load_trained_model_from_checkpoint
from transformers import BertTokenizer

# Helpers

In [ ]:
def display_all(df, num_rows=10000, num_cols=10000, col_width=-1):
    with pd.option_context('display.max_rows', num_rows, 'display.max_columns', num_cols, 'display.max_colwidth', -1):
        display(df)
        
def init_tokenizer_and_load_bert_model(model_dir, model_name, model_trainable=True, lowercase=True):

    vocab_path = f'{model_dir}/vocab.txt'
    config_path = f'{model_dir}/bert_config.json'
    checkpoint_path = f'{model_dir}/bert_model.ckpt'
    
    tokenizer = BertTokenizer(vocab_path, do_lower_case=lowercase)
    model = load_trained_model_from_checkpoint(config_path, checkpoint_path, trainable=model_trainable)
    
    print('vocab_size:', len(tokenizer.vocab))
    print('loaded model: ', model_name)
    
    return tokenizer, model

def get_bert_vectors(tokenizer, df, col_name, vector_length=512):
    tokenize = lambda sentence: tokenizer.encode_plus(sentence, max_length=vector_length, padding='max_length', truncation=True)
    df[f'{col_name}_tokens'] = df[col_name].map(tokenize)

    df[f'{col_name}_input_ids'] = df[f'{col_name}_tokens'].map(lambda x: x['input_ids'])
    df[f'{col_name}_token_type_ids'] = df[f'{col_name}_tokens'].map(lambda x: x['token_type_ids'])
    df[f'{col_name}_attention_mask'] = df[f'{col_name}_tokens'].map(lambda x: x['attention_mask'])
    
    input_ids = np.stack(df[f'{col_name}_input_ids'])
    token_type_ids = np.stack(df[f'{col_name}_token_type_ids'])
    attention_mask = np.stack(df[f'{col_name}_attention_mask'])
    vectors = {'input_ids': input_ids, 'token_type_ids': token_type_ids, 'attention_mask': attention_mask}
    
    return vectors

def bert_predict_in_batches(vectors, num_batches, output_shape):

  vector_input_ids_batches = np.array_split(vectors['input_ids'], num_batches)
  vector_token_type_ids_batches = np.array_split(vectors['token_type_ids'], num_batches)
  vector_attention_mask_batches = np.array_split(vectors['attention_mask'], num_batches)

  X = np.array([]).reshape((0, output_shape))

  input_vectors = zip(vector_input_ids_batches, vector_token_type_ids_batches, vector_attention_mask_batches)
  for input_ids, token_type_ids, attention_mask in input_vectors:
    all_vectors = (input_ids, token_type_ids, attention_mask)
    predictions = bert_model.predict(all_vectors, verbose=1)

    X_batch = predictions[:, 0, :]
    print('current predictions shape: ', X_batch.shape)
    X = np.concatenate([X, X_batch])
    print('all predictions shape: ', X.shape)

  return X

# Reading data

In [ ]:
train_fake = pd.read_csv(train_dir)
train_fake['is_fake'] = train_fake['is_fake'].astype('int8')
test_fake = pd.read_csv(test_dir)

train_fake.shape, test_fake.shape

Some fields have null values - temporarily we are filing them with nans to be able to get predictions.

In [ ]:
train_fake.fillna('unknown', inplace=True)
test_fake.fillna('unknown', inplace=True)

# Bert model

In [ ]:
tokenizer, bert_model = init_tokenizer_and_load_bert_model(model_dir, bert_model_name, model_trainable=True, lowercase=uncased)

## tokenize

In [ ]:
%%time
train_title_vectors = get_bert_vectors(tokenizer, train_fake, 'title', vector_length=512)
[vec.shape for vec in train_title_vectors.values()]

In [ ]:
%%time
train_text_vectors = get_bert_vectors(tokenizer, train_fake, 'text', vector_length=512)
[vec.shape for vec in train_text_vectors.values()]

In [ ]:
%%time
test_title_vectors = get_bert_vectors(tokenizer, test_fake, 'title', vector_length=512)
[vec.shape for vec in test_title_vectors.values()]

In [ ]:
%%time
test_text_vectors = get_bert_vectors(tokenizer, test_fake, 'text', vector_length=512)
[vec.shape for vec in test_text_vectors.values()]

## bert predict and save

In [ ]:
bert_output_shape = bert_model.layers[-1].output_shape[2]

In [ ]:
train_X_title = bert_predict_in_batches(train_title_vectors, 5, bert_output_shape)
np.save(f'{out_dir}/train_X_title_{bert_model_name}.npy', train_X_title)

In [ ]:
train_X_text = bert_predict_in_batches(train_text_vectors, 15, bert_output_shape)
np.save(f'{out_dir}/train_X_text_{bert_model_name}.npy', train_X_text)

In [ ]:
test_X_title = bert_predict_in_batches(test_title_vectors, 5, bert_output_shape)
np.save(f'{out_dir}/test_X_title_{bert_model_name}.npy', test_X_title)

In [ ]:
test_X_text = bert_predict_in_batches(test_text_vectors, 15, bert_output_shape)
np.save(f'{out_dir}/test_X_text_{bert_model_name}.npy', test_X_text)